In [1]:
#HIDE
try:
    import setup
except:
    pass
from notebook_helper import *
notebook_setup()
from modules.la import get_la_df, get_la_with_leagues

## Political control

This uses data from http://opencouncildata.co.uk/downloads.php to get the overal control and the current coalition for Uk councils, mapped against out groupings and leagues. 

In [2]:
cdf = pd.read_csv(Path("data", "source", "opencouncildata_councils.csv"))
cdf["name"] = cdf["name"].str.replace(" CC","").str.strip()
cdf = cdf.la.create_code_column(source_col="name")
cdf = cdf.set_index("local-authority-code")
cdf = cdf[["majority", "coalition"]]

In [3]:

# match for new Northamptonshire councils
adf = pd.DataFrame([["WNT", "CON", "CON"], ["NNT", "CON", "CON"]],
                   columns=["local-authority-code", "majority", "coaliton"]).set_index("local-authority-code")

cdf = pd.concat([cdf, adf])


In [4]:
df = (get_la_with_leagues()
      .set_index("local-authority-code")
      .join(cdf, how="outer")
      .reset_index())


In [5]:
df.pivot_table("local-authority-code", index="majority", columns="league-group", aggfunc="count").fillna(0)


league-group,County councils,District councils,Northern Ireland,Single tier
majority,,,,
CON,21.0,89.0,0.0,33.0
IND,0.0,4.0,0.0,7.0
LAB,0.0,18.0,0.0,66.0
LD,0.0,21.0,0.0,4.0
NOC,3.0,49.0,11.0,71.0
PC,0.0,0.0,0.0,1.0


This source does not have information for combined authorities, and there's not many of them, so we'll ignore those.

Similarly, given the low number of NI with plans, we'll ignore political control in NI. 

This means political control will be avaliable for 'County councils', 'District councils' and 'Single tier'.

Reviewing the caolition information, there is too much information avalaible to put it in an easy drop down. 

In [6]:
mask = df["league-group"].isin(["County councils",
                               "District councils", "Single tier"])
df = df[mask]
df["majority"] = df["majority"].map({"CON": "Conservative",
                                     "IND": "Independent",
                                     "LAB": "Labour",
                                     "LD": "Liberal Democrat",
                                     "NOC": "Coalition/Minority",
                                     "PC": "Plaid Cymru"})

df.pivot_table("local-authority-code", index="majority",
               columns="league-group", aggfunc="count").fillna(0)


league-group,County councils,District councils,Single tier
majority,,,
Coalition/Minority,3.0,49.0,71.0
Conservative,21.0,89.0,33.0
Independent,0.0,4.0,7.0
Labour,0.0,18.0,66.0
Liberal Democrat,0.0,21.0,4.0
Plaid Cymru,0.0,0.0,1.0


In [8]:
# export this data
df = df.rename(columns={"majority":"political_control"})
cols = ["local-authority-code","official-name", "league-group", "political_control"]
df[cols].to_csv(Path("data", "outputs", "political_control.csv"), index=False)